In [1]:
import json
import pandas as pd
import itertools
from collections import Counter

In [3]:
train = pd.read_json('../data/train.json', encoding='utf-8')

In [4]:
validation = pd.read_json('../data/val.json', encoding='utf-8')

In [15]:
song_meta = pd.read_json('../data/song_meta.json', encoding='utf-8')
song_meta.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [11]:
with open('../data/genre_gn_all.json', encoding='utf-8') as f:
    genre = json.load(f)
    
genre

{'GN0100': '발라드',
 'GN0101': '세부장르전체',
 'GN0102': "'80",
 'GN0103': "'90",
 'GN0104': "'00",
 'GN0105': "'10-",
 'GN0200': '댄스',
 'GN0201': '세부장르전체',
 'GN0202': "'80",
 'GN0203': "'90",
 'GN0204': "'00",
 'GN0205': "'10-",
 'GN0300': '랩/힙합',
 'GN0301': '세부장르전체',
 'GN0302': '랩 스타일',
 'GN0303': '보컬 스타일',
 'GN0304': '언더그라운드 힙합',
 'GN0305': '시대별',
 'GN0400': 'R&B/Soul',
 'GN0401': '세부장르전체',
 'GN0402': '어반',
 'GN0403': 'R&B',
 'GN0500': '인디음악',
 'GN0501': '세부장르전체',
 'GN0502': '포크',
 'GN0503': '록',
 'GN0504': '일렉',
 'GN0505': '힙합',
 'GN0506': '발라드',
 'GN0507': "'90",
 'GN0508': "'00",
 'GN0509': "'10-",
 'GN0600': '록/메탈',
 'GN0601': '세부장르전체',
 'GN0602': "'70",
 'GN0603': "'80",
 'GN0604': "'90",
 'GN0605': "'00",
 'GN0606': "'10-",
 'GN0700': '성인가요',
 'GN0701': '세부장르전체',
 'GN0702': '신세대트로트',
 'GN0703': '전설의트로트',
 'GN0704': '뽕짝트로트',
 'GN0705': '트로트메들리',
 'GN0706': "트로트'60-'70",
 'GN0707': "트로트'80-'90",
 'GN0708': "트로트'00-",
 'GN0709': "성인가요'80-'90",
 'GN0710': "성인가요'00-",
 'GN0800': '포크/블루스',

In [13]:
genre = {key: (genre[key[:-1]+'0'] if key[-2:] =='01' else value) for key, value in genre.items()}
genre

{'GN0100': '발라드',
 'GN0101': '발라드',
 'GN0102': "'80",
 'GN0103': "'90",
 'GN0104': "'00",
 'GN0105': "'10-",
 'GN0200': '댄스',
 'GN0201': '댄스',
 'GN0202': "'80",
 'GN0203': "'90",
 'GN0204': "'00",
 'GN0205': "'10-",
 'GN0300': '랩/힙합',
 'GN0301': '랩/힙합',
 'GN0302': '랩 스타일',
 'GN0303': '보컬 스타일',
 'GN0304': '언더그라운드 힙합',
 'GN0305': '시대별',
 'GN0400': 'R&B/Soul',
 'GN0401': 'R&B/Soul',
 'GN0402': '어반',
 'GN0403': 'R&B',
 'GN0500': '인디음악',
 'GN0501': '인디음악',
 'GN0502': '포크',
 'GN0503': '록',
 'GN0504': '일렉',
 'GN0505': '힙합',
 'GN0506': '발라드',
 'GN0507': "'90",
 'GN0508': "'00",
 'GN0509': "'10-",
 'GN0600': '록/메탈',
 'GN0601': '록/메탈',
 'GN0602': "'70",
 'GN0603': "'80",
 'GN0604': "'90",
 'GN0605': "'00",
 'GN0606': "'10-",
 'GN0700': '성인가요',
 'GN0701': '성인가요',
 'GN0702': '신세대트로트',
 'GN0703': '전설의트로트',
 'GN0704': '뽕짝트로트',
 'GN0705': '트로트메들리',
 'GN0706': "트로트'60-'70",
 'GN0707': "트로트'80-'90",
 'GN0708': "트로트'00-",
 'GN0709': "성인가요'80-'90",
 'GN0710': "성인가요'00-",
 'GN0800': '포크/블루스',
 'GN0801': '

In [37]:
genre_word_id = dict()
genre_id_word = dict()

for i, t in enumerate(genre.values()):
    genre_word_id[t] = i
    genre_id_word[i] = t
    
print(f"genre word id: {genre_word_id}")
print()
print(f"genre id word: {genre_id_word}")

genre word id: {'발라드': 230, "'80": 96, "'90": 97, "'00": 107, "'10-": 108, '댄스': 231, '랩/힙합': 229, '랩 스타일': 14, '보컬 스타일': 15, '언더그라운드 힙합': 16, '시대별': 236, 'R&B/Soul': 169, '어반': 104, 'R&B': 21, '인디음악': 23, '포크': 111, '록': 166, '일렉': 26, '힙합': 27, '록/메탈': 66, "'70": 75, '성인가요': 40, '신세대트로트': 41, '전설의트로트': 42, '뽕짝트로트': 43, '트로트메들리': 44, "트로트'60-'70": 45, "트로트'80-'90": 46, "트로트'00-": 47, "성인가요'80-'90": 48, "성인가요'00-": 49, '포크/블루스': 51, "'60-'70": 114, "'80-'90": 174, 'POP': 165, '얼터너티브팝': 58, '올디스': 59, '월드팝': 60, '모던록': 67, '얼터너티브록': 68, '프로그레시브/아트록': 69, '하드록': 70, '헤비메탈': 71, '뉴메탈/하드코어': 72, '포스트록': 73, "'60": 74, '일렉트로니카': 233, '하우스': 239, '클럽뮤직': 84, '일렉트로닉팝': 85, '팝랩': 92, '얼터너티브힙합': 93, '갱스터/하드코어랩': 94, 'East&West': 95, '컨템포러리 R&B': 102, '소울': 103, '포크/블루스/컨트리': 110, '블루스': 112, '컨트리': 113, "'00-": 116, 'OST': 118, '국내영화': 119, '국외영화': 120, '국내드라마': 121, '국외드라마': 122, '애니메이션/게임': 123, '국내뮤지컬': 251, '국외뮤지컬': 252, '애니메이션/웹툰': 126, '게임': 127, '클래식': 129, '관현악': 130, '교향곡': 131, '실내악':

In [17]:
song_meta['genre_list'] = song_meta.song_gn_dtl_gnr_basket
song_meta['song_gn_word'] = song_meta.genre_list.apply(lambda e: list(map(lambda x: genre[x], filter(lambda y: y in genre, e))))
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,genre_list,song_gn_word
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,[GN0901],[POP]
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,"[GN1601, GN1606]","[클래식, 독주곡]"
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,[GN0901],[POP]
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,"[GN1102, GN1101]","[일렉트로니카, 일렉트로니카]"
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,"[GN1802, GN1801]","[이지리스닝, 뉴에이지]"
...,...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,[GN2001],[월드뮤직]
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,[GN0901],[POP]
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,"[GN0105, GN0101]","['10-, 발라드]"
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,"[GN1807, GN1801]","[컴필레이션, 뉴에이지]"


In [66]:
temp = song_meta.loc[0:1]
temp.song_gn_word.apply(lambda x: list(genre_word_id[k] for k in x))

0         [165]
1    [129, 134]
Name: song_gn_word, dtype: object

In [67]:
song_meta['song_gn_code'] = song_meta.song_gn_word.apply(lambda x: list(genre_word_id[k] for k in x))
song_meta.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,genre_list,song_gn_word,song_gn_word_split,song_gn_code
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,[GN0901],[POP],[POP],[165]
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,"[GN1601, GN1606]","[클래식, 독주곡]","[클래식, 독주곡]","[129, 134]"
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,[GN0901],[POP],[POP],[165]
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,"[GN1102, GN1101]","[일렉트로니카, 일렉트로니카]","[일렉트로니카, 일렉트로니카]","[233, 233]"
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,"[GN1802, GN1801]","[이지리스닝, 뉴에이지]","[이지리스닝, 뉴에이지]","[157, 171]"


In [20]:
song_meta['song_gn_word_split'] = song_meta.song_gn_word.apply(lambda e: list(itertools.chain(*map(lambda x: x.split('/'), e))))

In [69]:
song_meta_dropna = song_meta.dropna(subset=['song_gn_word_split'])

# song_id_genre = dict(zip(song_meta_dropna['id'], song_meta_dropna['song_gn_word_split']))
song_id_genre = dict(zip(song_meta['id'], song_meta['song_gn_code']))
song_id_genre

{0: [165],
 1: [129, 134],
 2: [165],
 3: [233, 233],
 4: [157, 171],
 5: [172],
 6: [129, 130, 142],
 7: [129, 130, 142],
 8: [229],
 9: [108, 230],
 10: [229],
 11: [169],
 12: [172],
 13: [172, 147],
 14: [241, 84, 239, 233],
 15: [129, 162],
 16: [165],
 17: [228, 108, 226, 231, 231],
 18: [247, 243],
 19: [108, 23, 111, 51, 108],
 20: [220, 129],
 21: [],
 22: [174, 165, 59],
 23: [129, 143, 138],
 24: [233, 233],
 25: [229],
 26: [165],
 27: [166, 164],
 28: [165],
 29: [119, 118],
 30: [227, 108, 226, 231, 118, 231, 121],
 31: [162, 171],
 32: [118, 121],
 33: [233, 234],
 34: [233, 233],
 35: [108, 23, 111, 51, 108],
 36: [126, 123, 118],
 37: [172, 152],
 38: [172],
 39: [169, 108, 102],
 40: [68, 66],
 41: [172, 147, 146],
 42: [69, 107, 66],
 43: [112, 110],
 44: [72, 107, 66],
 45: [177, 184],
 46: [66],
 47: [108, 23, 16, 27, 229],
 48: [51, 174],
 49: [108, 230],
 50: [97, 68, 66],
 51: [244, 243],
 52: [169, 103],
 53: [189, 188],
 54: [230],
 55: [233],
 56: [165, 58, 6

In [22]:
genre_count = dict(Counter([__ for _ in song_meta.song_gn_word_split for __ in _]))
tag_count = Counter([__ for _ in train.tags for __ in _])

In [23]:
print('태그 내에 들어있는 장르 이름들 :', set(tag_count).intersection(genre_count))

태그 내에 들어있는 장르 이름들 : {'파두', '하드코어랩', '아트록', '영어동화', '갱스터', '얼터너티브팝', '교과서동요', '록', '국외영화', '팝', '켈틱', '발레', '독주곡', 'ASMR', '재즈', '성악', '일렉트로니카', '하드코어', '애시드', '어린이', '탱고', '댄스', '아이돌', '기타', '국악가요', '발라드', '만화', '얼터너티브록', '보사노바', '어반', '뉴에이지', '요가', '플라멩코', '컴필레이션', '일렉트로닉팝', '연주곡', '게임', '기능성음악', '성가', 'CCM', '합창곡', '뮤지컬', '하우스', '중국음악', '하드록', '이지리스닝', '현대음악', '모던록', '무용', '퓨전', '찬송가', '랩', '자연', '시부야케이', '포스트록', '칸초네', '라틴재즈', '헤비메탈', 'OST', '오페라', '스윙', '태교', '레게', '국내영화', '컨트리', '종교음악', '협주곡', '어린이클래식', '올디스', '샹송', '워십', '불교', '월드뮤직', '소울', '포크', '라틴', '힐링', '국내드라마', '클럽', '관현악', '사물놀이', '국외드라마', '뮤직테라피', '자장가', '성인가요', '프렌치팝', '국외뮤지컬', '집중력', '일렉', '피아노', '프로그레시브', '팝랩', '얼터너티브힙합', '클럽뮤직', '애니메이션', '월드팝', 'EDM', '반려동물', '브라질', '인디음악', '숙면유도', '빅밴드', '판소리', '블루스', '전래동요', '뉴메탈', '메탈', '민요', '영어동요', 'DJ', '크로스오버', '힙합', '명상', '웹툰', '창작동요', '교향곡', '보컬재즈', '실내악', '국악', '클래식'}


In [72]:
train['genre'] = train.songs.apply(lambda e: Counter(itertools.chain(*list(map(lambda x: song_id_genre[x], e)))).most_common())
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,genre
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,"[(165, 10), (66, 10), (58, 5), (107, 4), (111,..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[(108, 16), (230, 11), (66, 11), (107, 10), (2..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[(108, 18), (169, 15), (23, 12), (21, 10), (66..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,"[(108, 20), (230, 17), (23, 8), (172, 6), (165..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,"[(231, 55), (97, 34), (226, 15), (227, 8), (10..."


In [75]:
import numpy as np

genre_onehot = np.zeros((len(train), len(genre_id_word)))
genre_onehot.shape

(115071, 254)

In [113]:
from tqdm.auto import tqdm

for idx, row in tqdm(enumerate(train.iterrows()), total=train.shape[0]):
#     if idx == 0:
#         print(row[-1].genre)
#     else:
#         break
    hot = sorted(dict(row[1].genre).keys())
    genre_onehot[idx, hot] = 1
    
genre_onehot[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [116]:
np.save('genre_onehot.npy', genre_onehot)

In [190]:
pd.options.display.max_colwidth = 1000

In [191]:
train[['genre', 'tags']]

,genre,tags
0,"[(POP, 10), (록, 10), (메탈, 10), (포크, 6), (얼터너티브팝, 5), ('00, 4), (블루스, 3), (컨트리, 3), (일렉트로니카, 3), (얼터너티브록, 2), ('90, 1), (하드록, 1), (어반, 1), (R&B, 1), (Soul, 1), (컨템포러리 R&B, 1), (월드팝, 1), ('00-'10, 1), (J-POP, 1)]",[락]
1,"[(록, 16), ('10-, 16), (발라드, 11), (메탈, 11), ('00, 10), (인디음악, 7), (재즈, 7), (포크, 4), (J-Jazz, 4), (크로스오버, 4), (클래식, 4), (OST, 4), (J-Newage, 3), (뉴에이지, 3), (국내드라마, 3), (빅밴드, 3), (스윙, 3), (블루스, 2), ('90, 1), (일렉트로니카, 1), (팝 스타일, 1), (댄스 스타일, 1), (국내영화, 1), (POP, 1)]","[추억, 회상]"
2,"[(R&B, 25), ('10-, 18), (Soul, 15), (인디음악, 12), (록, 8), (메탈, 4), (보컬 스타일, 3), (랩, 3), (힙합, 3), (포크, 3), (발라드, 2), (블루스, 2), (어반, 1), (재즈, 1), (애시드, 1), (퓨전, 1), (팝, 1), (일렉트로니카, 1), (팝 스타일, 1), (일렉, 1)]","[까페, 잔잔한]"
3,"[('10-, 20), (발라드, 17), (인디음악, 8), (록, 8), (애니메이션, 6), (재즈, 6), (POP, 5), (OST, 5), (R&B, 5), (Soul, 5), (컨템포러리 R&B, 5), (메탈, 4), (웹툰, 3), (만화, 3), (게임, 3), (포크, 3), (J-Jazz, 2), (국내드라마, 2), (블루스, 2), ('90, 1), ('00, 1), (J-Newage, 1), (뉴에이지, 1), (보컬재즈, 1), (빅밴드, 1), (스윙, 1), (신세대트로트, 1), (트로트'00-, 1), (성인가요, 1)]","[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스, 겨울왕국, 크리스마스송]"
4,"[(댄스, 55), ('90, 34), (아이돌, 15), (남자 아이돌, 8), ('00, 8), (여자 아이돌, 7), (발라드, 6), (랩, 5), (힙합, 5), (보컬 스타일, 3), (R&B, 2), (Soul, 1), (랩 스타일, 1), (록, 1), (메탈, 1)]",[댄스]
5,"[(일렉트로니카, 19), (클럽뮤직, 11), (POP, 7), (하우스, 6), (DJ, 4), (랩, 3), (힙합, 3), (일렉트로닉팝, 3), (R&B, 2), (Soul, 2), (컨템포러리 R&B, 2), (어반, 1), (얼터너티브팝, 1), (록, 1), (메탈, 1), (국외영화, 1), (OST, 1), (클럽, 1)]","[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌드, 일렉]"
6,"[(발라드, 33), ('10-, 29), (R&B, 8), (OST, 8), (국내드라마, 8), ('00, 8), (포크, 6), (인디음악, 5), (Soul, 4), (록, 4), (블루스, 3), (남자 아이돌, 3), (아이돌, 3), (메탈, 2), (어반, 1)]","[짝사랑, 취향저격, 슬픔, 고백, 사랑, 이별]"
7,"[(발라드, 12), ('00, 11), (록, 3), (메탈, 3), (댄스, 3), (아이돌, 2), (보컬 스타일, 2), (랩, 2), (힙합, 2), (R&B, 2), (남자 아이돌, 1), (여자 아이돌, 1), (Soul, 1), (국내영화, 1), (OST, 1), ('90, 1)]","[잔잔한, 추억, 회상]"
8,"[(댄스, 31), ('10-, 22), ('00, 11), (록, 10), (메탈, 10), (아이돌, 8), (발라드, 5), (남자 아이돌, 4), (여자 아이돌, 4), (성인가요, 1), (일렉트로니카, 1), (팝 스타일, 1), (댄스 스타일, 1), (OST, 1), (국내드라마, 1), ('90, 1)]","[일렉트로니카, 포크, 메탈, 락, 댄스, 인디]"
9,"[(록, 20), ('10-, 16), (메탈, 15), (인디음악, 5), ('00, 4)]","[록, Metal, 이일우, M에센셜, 메탈, Rock, 락]"
